## A Portfolio of Technology Stocks in the S&P 500 using Markowitz's Modern Portfolio Theory

The objective of the project is look for the optimal portfolio which give best risk-return trade-off that lie along the minimum variance frontier within a set of given risky assets.

The assets in this case are the Technology Stocks of the S&P 500

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy import optimize
import warnings
warnings.filterwarnings('ignore')

<br>

**The Historical stock datasets was downloaded in the form of cvs files from Yahoo Finanace and in the cell below, the data was read inton a Pandas dataframe**

**A much faster approach is an API call, one of which is the use of a library such as yfinance to get the stock data from Yahoo finance**

In [2]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

combined_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    updated_df = df[['Date', 'Close']]
    updated_df[file_name] = updated_df['Close']
    updated_df.drop(['Close'], axis=1, inplace=True)
    updated_df['Date'] = pd.to_datetime(arg=updated_df['Date'])
    updated_df.set_index('Date', inplace=True)
    
    combined_df = pd.concat([combined_df, updated_df], axis=1)

    
# While downloading the dataset, the ending dates considered for the stock was 30th of August but some
# stocks had their end dates beyond that.The line of code was used to make the stock end dates uniform
combined_df.drop([combined_df.index[-1], combined_df.index[-2]], inplace=True)


# Reversing the order of the sorting

<br>
<br>

**Viewing the dataset**

In [3]:
combined_df

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANET,ANSS,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,14.686786,52.950001,28.570000,36.029999,30.809999,32.930000,10.700000,5.480000,NaN,57.389999,...,16.820000,55.820000,31.480000,13.750000,20.815001,29.760000,30.340000,35.820000,30.980000,35.720001
2012-01-04,14.765714,52.930000,28.280001,35.980000,30.209999,32.439999,10.760000,5.460000,NaN,56.400002,...,16.780001,55.790001,31.570000,14.180000,20.105000,29.570000,30.360001,35.820000,31.299999,35.450001
2012-01-05,14.929643,51.919998,28.480000,36.139999,30.820000,32.430000,10.830000,5.460000,NaN,57.299999,...,17.280001,55.570000,31.920000,14.620000,20.385000,29.780001,31.090000,35.740002,32.759998,35.400002
2012-01-06,15.085714,51.830002,28.719999,35.900002,30.959999,32.759998,11.010000,5.430000,NaN,56.939999,...,17.150000,55.580002,32.360001,14.630000,20.075001,29.780001,31.809999,35.400002,33.490002,35.110001
2012-01-09,15.061786,51.740002,28.530001,36.599998,31.080000,32.610001,11.340000,5.590000,NaN,56.680000,...,17.809999,55.220001,32.810001,14.890000,20.870001,30.160000,31.559999,35.020000,33.750000,34.950001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,176.380005,314.140015,512.429993,173.690002,208.600006,101.650002,142.520004,101.800003,179.190002,298.559998,...,104.230003,406.000000,127.559998,102.110001,52.259998,165.589996,375.119995,202.570007,39.869999,265.920013
2023-08-25,178.610001,318.760010,525.059998,178.160004,215.919998,102.820000,144.360001,102.250000,180.899994,304.890015,...,105.389999,408.549988,128.639999,103.779999,52.150002,167.809998,378.790009,205.740005,39.490002,268.390015
2023-08-28,180.190002,320.910004,529.919983,180.330002,217.089996,103.220001,145.419998,102.610001,181.960007,308.149994,...,106.199997,416.130005,129.750000,104.570000,53.259998,168.699997,384.250000,206.529999,41.410000,269.920013


<br>
<br>
<br>

The historical stock data considered spanned ten years, using daily data - from `01-01-2012` to `30-08-2023`

The start date for stock was `03-01-2012` as the first two days of 2012 were public holidays.

According to Google, there are approximately `252 days in a year`, `21 days per month` and `12 months per year` for trading in a Stock market.

Some Technology companies on the S&P 500 do not have their stock options extend back to 2012, and they were therfore dropped from the dataset used leaving us with 52 portfolios

In [4]:
# Dropping portfolios that are not up to 10 years

combined_df.dropna(axis=1, how='any', inplace=True)

In [5]:
combined_df.sort_values(['Date'], ascending=False)

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-30,187.649994,323.250000,545.359985,181.570007,221.550003,104.639999,150.949997,106.589996,315.959991,892.280029,...,107.980003,420.070007,131.929993,107.370003,54.250000,169.229996,397.959991,209.119995,42.520000,279.109985
2023-08-29,184.119995,323.450012,540.570007,182.559998,219.399994,104.400002,149.979996,105.919998,314.209991,889.929993,...,107.779999,417.910004,131.190002,107.370003,54.240002,170.690002,391.589996,207.350006,41.549999,273.489990
2023-08-28,180.190002,320.910004,529.919983,180.330002,217.089996,103.220001,145.419998,102.610001,308.149994,861.080017,...,106.199997,416.130005,129.750000,104.570000,53.259998,168.699997,384.250000,206.529999,41.410000,269.920013
2023-08-25,178.610001,318.760010,525.059998,178.160004,215.919998,102.820000,144.360001,102.250000,304.890015,851.820007,...,105.389999,408.549988,128.639999,103.779999,52.150002,167.809998,378.790009,205.740005,39.490002,268.390015
2023-08-24,176.380005,314.140015,512.429993,173.690002,208.600006,101.650002,142.520004,101.800003,298.559998,854.460022,...,104.230003,406.000000,127.559998,102.110001,52.259998,165.589996,375.119995,202.570007,39.869999,265.920013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-09,15.061786,51.740002,28.530001,36.599998,31.080000,32.610001,11.340000,5.590000,56.680000,30.219999,...,17.809999,55.220001,32.810001,14.890000,20.870001,30.160000,31.559999,35.020000,33.750000,34.950001
2012-01-06,15.085714,51.830002,28.719999,35.900002,30.959999,32.759998,11.010000,5.430000,56.939999,28.309999,...,17.150000,55.580002,32.360001,14.630000,20.075001,29.780001,31.809999,35.400002,33.490002,35.110001
2012-01-05,14.929643,51.919998,28.480000,36.139999,30.820000,32.430000,10.830000,5.460000,57.299999,28.490000,...,17.280001,55.570000,31.920000,14.620000,20.385000,29.780001,31.090000,35.740002,32.759998,35.400002


<br>
<br>

# Exploratory Data Analysis

**Viewing the info of the dataset to ensure that the available data is consistent**

In [ ]:
combined_df.info()

# Implementation of the Markowitz's Modern Portfolio Theory

52 Portfolios of Technology companies from the S&P 500 are being considered

#### Important parameters


`Expected return for a portfolio` = `Transpose of weights of the assets within the portfolio` * `Expected return`

`StandardDeviation` or `Volatility` = squar root of (`Transpose of Weights of assets` * `variance-covariance` * `Weights of assests`)

`Risk free rate` is a return an investor can expect from an investment, given the risk associated is zero.

`Sharpe ratio` is a measure of risk-adjusted return

`Sharpe ratio` = (`ExpectedReturn` - `RiskFreeRate`) / `Variance`



<br>

### Daily Returns

<br>

The returns using the daily dataset was obtained for the individual portfolio.

In [6]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

daily_returns = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    if file_name in list(combined_df.columns):
        ind_daily_return = pd.DataFrame(combined_df[file_name].pct_change())
        daily_returns = pd.concat([daily_returns, ind_daily_return], axis=1)

In [7]:
# viewing the origin dataframe after calculating daily returns

daily_returns

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.005374,-0.000378,-0.010150,-0.001388,-0.019474,-0.014880,0.005607,-0.003650,-0.017250,-0.017271,...,-0.002378,-0.000537,0.002859,0.031273,-0.034110,-0.006384,0.000659,0.000000,0.010329,-0.007559
2012-01-05,0.011102,-0.019082,0.007072,0.004447,0.020192,-0.000308,0.006506,0.000000,0.015957,0.001406,...,0.029797,-0.003943,0.011086,0.031030,0.013927,0.007102,0.024045,-0.002233,0.046645,-0.001410
2012-01-06,0.010454,-0.001733,0.008427,-0.006641,0.004542,0.010176,0.016620,-0.005495,-0.006283,-0.006318,...,-0.007523,0.000180,0.013784,0.000684,-0.015207,0.000000,0.023159,-0.009513,0.022283,-0.008192
2012-01-09,-0.001586,-0.001736,-0.006616,0.019498,0.003876,-0.004579,0.029973,0.029466,-0.004566,0.067467,...,0.038484,-0.006477,0.013906,0.017772,0.039601,0.012760,-0.007859,-0.010735,0.007763,-0.004557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,-0.026170,-0.003805,-0.034444,-0.021354,0.020748,-0.007712,-0.037027,-0.069725,-0.014100,-0.025234,...,-0.027977,-0.013941,-0.010933,-0.030479,-0.018038,-0.024966,-0.013673,-0.012769,-0.028745,-0.015439
2023-08-25,0.012643,0.014707,0.024647,0.025736,0.035091,0.011510,0.012910,0.004420,0.021202,-0.003090,...,0.011129,0.006281,0.008467,0.016355,-0.002105,0.013407,0.009784,0.015649,-0.009531,0.009289
2023-08-28,0.008846,0.006745,0.009256,0.012180,0.005419,0.003890,0.007343,0.003521,0.010692,0.010871,...,0.007686,0.018553,0.008629,0.007612,0.021285,0.005304,0.014414,0.003840,0.048620,0.005701


In [8]:
# This was written to view the dataframe from the current to the earliest dates, that is, reversing the order of the dataframe
# And the multiplication by 100 was done to express the returns as percentages which is the required way to express them

daily_returns = daily_returns.sort_values(['Date'], ascending=False).mul(100)
daily_returns

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-30,1.917227,-0.061837,0.886098,-0.542283,0.979949,0.229882,0.646754,0.632551,0.556952,0.264070,...,0.185567,0.516858,0.564061,0.000000,0.018433,-0.855355,1.626700,0.853624,2.334539,2.054918
2023-08-29,2.181027,0.791502,2.009742,1.236620,1.064074,1.143190,3.135743,3.225803,1.966574,3.350441,...,1.487761,0.427751,1.109828,2.677635,1.840038,1.179612,1.910214,0.397040,0.338080,1.322606
2023-08-28,0.884609,0.674487,0.925606,1.218005,0.541866,0.389030,0.734273,0.352079,1.069231,1.087085,...,0.768572,1.855346,0.862874,0.761227,2.128468,0.530361,1.441430,0.383977,4.861985,0.570065
2023-08-25,1.264313,1.470680,2.464728,2.573552,3.509104,1.151006,1.291045,0.442040,2.120183,-0.308969,...,1.112919,0.628076,0.846661,1.635489,-0.210478,1.340662,0.978357,1.564890,-0.953090,0.928851
2023-08-24,-2.617044,-0.380536,-3.444448,-2.135449,2.074772,-0.771183,-3.702700,-6.972491,-1.410028,-2.523443,...,-2.797724,-1.394081,-1.093280,-3.047853,-1.803839,-2.496618,-1.367273,-1.276863,-2.874543,-1.543924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-09,-0.158614,-0.173645,-0.661553,1.949849,0.387600,-0.457866,2.997275,2.946593,-0.456619,6.746733,...,3.848391,-0.647717,1.390606,1.777170,3.960149,1.276021,-0.785916,-1.073452,0.776345,-0.455711
2012-01-06,1.045377,-0.173336,0.842693,-0.664076,0.454247,1.017570,1.662050,-0.549451,-0.628272,-0.631804,...,-0.752321,0.017999,1.378449,0.068399,-1.520721,0.000000,2.315854,-0.951315,2.228340,-0.819212
2012-01-05,1.110200,-1.908184,0.707210,0.444689,2.019202,-0.030823,0.650558,0.000000,1.595739,0.140594,...,2.979738,-0.394338,1.108647,3.102962,1.392688,0.710183,2.404476,-0.223333,4.664534,-0.141041


<br>
<br>

### Monthly returns for individual asset

<br>

The percentage returns per month for each portfolio.

**The `'M'` passed into the resample function was used to achieve this**

In [ ]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

monthly_returns = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    if file_name in list(combined_df.columns):
        ind_monthly_return = pd.DataFrame(combined_df[file_name].resample('M').ffill().pct_change())
        monthly_returns = pd.concat([monthly_returns, ind_monthly_return], axis=1)


In [ ]:
monthly_returns = monthly_returns.sort_values(['Date'], ascending=False).mul(100)
monthly_returns

## Averages and Variance

In [ ]:
daily_avg = pd.DataFrame(daily_returns.mean(), columns=['Average_Daily_Return'])
monthly_avg = pd.DataFrame(monthly_returns.mean(), columns=['Average_Monthly_Return'])


# Variance
daily_variance = pd.DataFrame(daily_returns.var(), columns=['Daily_Return_Variance'])
monthly_variance = pd.DataFrame(monthly_returns.var(), columns=['Monthly_Return_Variance'])

In [ ]:
averages = pd.concat([daily_avg, monthly_avg, daily_variance, monthly_variance], axis=1)

averages['Average_Annual_Return'] = pd.DataFrame(monthly_avg.mul(12))

averages['Annual_Variance'] = pd.DataFrame(monthly_variance.mul(12))

In [ ]:
averages

## Returns

In [ ]:
returns = pd.DataFrame(daily_returns.mean(), columns=['Returns']).mul(252)

returns

In [ ]:
returns = averages['Average_Annual_Return']

returns

## Variance-Covariance Matrix

In [ ]:
# Using the monthly returns, we compute the covariance matrix

covariance = monthly_returns.cov()
covariance

In [ ]:
# Using the monthly returns, we compute the covariance matrix

covariance2 = daily_returns.cov()
covariance2

## Equally-Weighted Portfolio

The sum of the individual weights is expected to be equal to 1

In [ ]:
def equal_weights(df):
    data = []
    weight = 1/len(list(df.columns))
    for col in df.columns:
        data.append([col, weight])
    
    weight_df = pd.DataFrame(data, columns=['Ticker', 'Weights'])
    
    
    return weight_df.set_index('Ticker')

In [ ]:
weight_df = equal_weights(combined_df)
weight_df

### Expected Returns

Expected return is the product of Weights and returns

In [ ]:
expected_return = weight_df.transpose().dot(returns)

expected_return.values[0][0]

### Expected Return in percentage

In [ ]:
expected_return_perc = expected_return['Returns'].apply('{:.2%}'.format)[0]

print(expected_return_perc)

## Standard Deviation

In [ ]:
std_dev = weight_df.transpose().dot(covariance).dot(weight_df).apply(np.sqrt,axis=1)

std_dev['Weights'].values[0]

### Sharpe Ratio

**Using a Risk Free Rate of `3.00%`**

In [ ]:
risk_free_rate = 3.8/100.0

sharpe_ratio = (expected_return['Returns'].values[0] - risk_free_rate)/std_dev['Weights'].values[0]

sharpe_ratio

### Optimal Risky Portfolio Weight

In [ ]:
def equal_weights(df):
    data = []
    weight = 1/len(list(df.columns))
    for col in df.columns:
        data.append([col, weight])
    
    weight_df = pd.DataFrame(data, columns=['Ticker', 'Weights'])
    
    
    return weight_df.set_index('Ticker')

In [ ]:
Optimal_weight_df = equal_weights(combined_df)

Optimal_weight_df

### Optimal Expected Returns

In [ ]:
optimal_expected_return = weight_df.transpose().dot(returns)

optimal_expected_return.values[0][0]

#### Optimal Expected Return in percentage

In [ ]:
optimal_expected_return_perc = optimal_expected_return['Returns'].apply('{:.2%}'.format)[0]

print(optimal_expected_return_perc)

#### Standard Deviation

In [ ]:
optimal_std_dev = weight_df.transpose().dot(covariance).dot(weight_df).apply(np.sqrt,axis=1)

optimal_std_dev['Weights'].values[0]

In [ ]:
optimal_std_dev_perc = optimal_std_dev['Weights'].apply('{:.2%}'.format)[0]

print(optimal_std_dev_perc)

### Sharpe Ratio

**Using a Risk Free Rate of `3.00%`**

In [ ]:
#set risk free asset rate of return
Rf=3.8  # August 2023 average risk  free rate of return in USA approx 3.8%
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
risk_free_rate = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(risk_free_rate)) 

In [ ]:
risk_free_rate = 3.8/100

sharpe_ratio = (expected_return['Returns'].values[0] - risk_free_rate)/std_dev['Weights'].values[0]

sharpe_ratio

In [ ]:
risk_free_rate

## Sharpe Ratio based Portfolio Optimization

### The Principle of duality

SInce Scipy Optimization Library only minimize, the principle of duality from optimization theory can be used to undertake transformation to obtain maximization.

In [ ]:
# Function to undertake Sharpe Ratio maximization subject to basic constraints of the portfolio

#dependencies

def MaximizeSharpeRatioOptmzn(AverageReturns, VarianceCovariancce, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    
    def  ObjectiveFunction(weights, AverageReturns, VarianceCovariancce, RiskFreeRate, PortfolioSize):
        Expected_Return = weights.transpose().dot(AverageReturns)
        Standard_Deviation = np.sqrt(weights.transpose().dot(VarianceCovariancce).dot(weights))
        funcDenom = Standard_Deviation
        funcNumer = Expected_Return - RiskFreeRate
        
        func = -(funcNumer / funcDenom)
        return func

    # define equality constraint representing fully invested portfolio
    def constraintEq(weights):
        A=np.ones(weights.shape)
        b=1
        constraintVal = np.matmul(A,weights.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    OptimizedSharpeRatio = optimize.minimize (ObjectiveFunction, x0 = xinit, args = (AverageReturns, VarianceCovariancce, 
                                                    RiskFreeRate, PortfolioSize), method = 'SLSQP', 
                                                     bounds = bnds, constraints = cons, tol = 10**-3)
    
    return OptimizedSharpeRatio

In [ ]:
portfolioSize = len(returns)

sharpeRatio = MaximizeSharpeRatioOptmzn(returns, covariance, risk_free_rate, portfolioSize)

In [ ]:
sharpeRatio

In [ ]:
pd.DataFrame(sharpeRatio)

In [ ]:
# obtain maximal Sharpe Ratio for k-portfolio 1 of Dow stocks

#set portfolio size
#portfolioSize = Columns

#set risk free asset rate of return
Rf=3.8  # August 2023 average risk  free rate of return in USA approx 3.8%
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
risk_free_rate = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(risk_free_rate)) 

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#compute maximal Sharpe Ratio and optimal weights
result = MaximizeSharpeRatioOptmzn(returns, covariance2, risk_free_rate, portfolioSize)
xOptimal.append(result.weights)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio   
xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)


#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ',
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet)
print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )